In [1]:
suppressPackageStartupMessages({
    library(openxlsx)
    library(clusterProfiler)
})

Obtain cell type markers from text files.

In [2]:
txt.files <- list.files(pattern = ".txt", path = "RA2", full.names = T)
meta_data <- strsplit(txt.files, split = "_")

celltypes <- c("b-cells", "cd4-tcells", "cd8-tcells", "dendritic-cells", "fibro1-cd55", "fibro2a-thy1",
              "fibro2b-thy1", "macrophage", "mast-cells", "nk-cells", "plasma-cells", "platelets", "tph-cells")

GeneSets <- setNames(lapply(txt.files, function(cell.type.file) {
    read.table(cell.type.file, header = T, stringsAsFactors = F)$x
}), nm = celltypes)
    
GeneSets

$`b-cells`
 [1] "LINC00926" "MS4A1"     "LTB"       "CD37"      "FAM129C"   "CD22"     
 [7] "CD52"      "CD19"      "CD79B"     "CD79A"     "RASGRP2"   "BIRC3"    
[13] "MEF2C"     "SELL"      "FCRL5"     "POU2F2"    "SPIB"     

$`cd4-tcells`
[1] "IL7R"  "LTB"   "CD52"  "ETS1"  "SELL"  "BIRC3" "TCF7"  "RPS29"

$`cd8-tcells`
 [1] "PTPRC"  "CD2"    "CXCR4"  "CD3D"   "CORO1A" "LCP1"   "CD52"   "IL32"  
 [9] "ARL4C"  "ITGAL"  "HCST"   "TMSB4X"

$`dendritic-cells`
 [1] "CD74"    "LYZ"     "CD83"    "AIF1"    "CTSH"    "GPX1"    "SAMHD1" 
 [8] "MS4A6A"  "CIITA"   "GRASP"   "FCN1"    "PLEKHO1" "RASSF4"  "TYROBP" 

$`fibro1-cd55`
[1] "MMP3"   "MMP1"   "FN1"    "CRTAC1" "HTRA1"  "CLU"    "PRG4"   "TIMP3" 

$`fibro2a-thy1`
 [1] "COL1A2"   "AEBP1"    "COL1A1"   "BGN"      "COL3A1"   "DCN"     
 [7] "COL6A3"   "COL6A1"   "COL5A2"   "FBLN1"    "DPT"      "SERPINF1"
[13] "COMP"     "FBN1"     "IGFBP7"   "TAGLN"    "C3"       "MXRA5"   
[19] "SPARC"    "THY1"    

$`fibro2b-thy1`
 [1] "BGN"    "IGFBP4" "AEBP1"  "COL1A2" "COL6A2" "C1R"    "CHI3L1" "DCN"   
 [9] "APOE"   "LUM"    "C3"     "COL6A3" "COL6A1" "CXCL12" "EFEMP1" "MMP2"  
[17] "PTGDS"  "DPT"    "VCAN"   "WISP2" 

$macrophage
 [1] "C1QA"     "FTL"      "RNASE1"   "PSAP"     "C1QB"     "CD14"    
 [7] "C1QC"     "MARCO"    "CTSD"     "GRN"      "CTSB"     "CD68"    
[13] "A2M"      "S100A9"   "FCER1G"   "SERPINA1" "GLUL"     "CCL18"   
[19] "STAB1"    "CTSZ"     "AIF1"     "CYBB"     "CTSS"     "ACP5"    
[25] "LYZ"      "DUSP1"    "SLCO2B1"  "CTSH"     "TYROBP"   "LGMN"    
[31] "MS4A6A"   "TYMP"     "LAPTM5"   "RASSF4"   "VAMP8"   

$`mast-cells`
 [1] "FOSB"   "BIRC3"  "SLC2A3" "NFKBIA" "CD83"   "NR4A1"  "BTG2"   "P2RX1" 
 [9] "IER2"   "SRGN"   "TPSAB1"

$`nk-cells`
[1] "CD247" "CD7"   "ITGAX"

$`plasma-cells`
 [1] "IGLL5"    "MZB1"     "IGLL1"    "DERL3"    "SSR4"     "PIM2"    
 [7] "SDC1"     "HERPUD1"  "TP53INP1" "CD79A"    "SLAMF7"   "JSRP1"   
[13] "SPCS3"    "SDF2L1"   "POU2AF1"  "HSP90B1"  "SEC11C"   "ITM2C"   
[19] "FCRL5"    "XBP1"    

$platelets
 [1] "COL4A1"  "COL4A2"  "PLVAP"   "A2M"     "ENG"     "CCL21"   "HSPG2"  
 [8] "COL18A1" "CRIP2"   "CD93"    "IGFBP7"  "FKBP1A"  "APLNR"   "CD59"   
[15] "PRCP"    "RAMP3"   "SPTBN1"  "NPDC1"   "VWF"     "TCF4"   

$`tph-cells`
 [1] "IL7R"    "LTB"     "IL2RG"   "CD2"     "CD3D"    "ETS1"    "CD7"    
 [8] "PTPRC"   "SLA"     "BIRC3"   "IL32"    "SPOCK2"  "SRGN"    "TNFRSF4"

# Run GO on celltype markers

We use the enricher() function from the [clusterProfiler](https://bioconductor.org/packages/release/bioc/html/clusterProfiler.html) R package to perform GO enrichment analysis. The gene annotations were obtained from the [MSigDb](http://software.broadinstitute.org/gsea/msigdb/collections.jsp) (GO Bological Processes dataset). Only terms with an adjusted p-value of >= 0.05 were retuerned.

In [3]:
h <- suppressPackageStartupMessages({read.gmt("../data/Patient1/c5.bp.v6.2.symbols.gmt")})

bp.eriched.terms <- setNames(lapply(celltypes, function(cl) {
    egmt <- enricher(GeneSets[[cl]], TERM2GENE = h, pvalueCutoff = 0.05)
    as.data.frame(egmt)
}), nm = celltypes)

Export results to excel file

In [4]:
library(openxlsx)

sh <- createWorkbook(title = "RA celltypes GO enrcihment")

for (cl in celltypes) {
    addWorksheet(wb = sh, sheetName = cl)
    firstrow <- paste0("RA2 cluster: ", gsub(pattern = "Cluster", replacement = "", x = cl))
    writeData(wb = sh, x = firstrow, sheet = cl)
    secondrow <- paste0("marker genes: ", paste(GeneSets[[cl]], collapse = ", "))
    writeData(wb = sh, x = secondrow, sheet = cl, startRow = 2)
    writeData(wb = sh, x = "", sheet = cl, startRow = 3)
    writeData(wb = sh, x = bp.eriched.terms[[cl]][, -1], sheet = cl, startRow = 4)
    setColWidths(wb = sh, sheet = cl, cols = 1:ncol(bp.eriched.terms[[cl]]), widths = "auto")
}

saveWorkbook(sh, file = "Cell_type_genes_for_GO/RA2_cells_clusters_GO_enrichment.xlsx", overwrite = TRUE)

# Date

In [5]:
date()

[1] "Mon Jun 10 09:40:00 2019"

# Session Info

In [6]:
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS  10.14.5

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats4    parallel  stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] GSEABase_1.42.0       graph_1.58.0          annotate_1.58.0      
 [4] XML_3.98-1.13         AnnotationDbi_1.44.0  IRanges_2.14.10      
 [7] S4Vectors_0.18.3      Biobase_2.40.0        BiocGenerics_0.26.0  
[10] clusterProfiler_3.8.1 openxlsx_4.1.0       

loaded via a namespace (and not attached):
 [1] viridis_0.5.1        tidyr_0.8.2          viridisLite_0.3.0   
 [4] bit64_0.9-7          jsonlite_1.6         splines_3.5.1       
 [7] ggraph_1.0.2         assertthat_0